## Preprocessing

In [1]:
from shutil import rmtree

In [2]:
rmtree('untitled_project')

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df = application_df.drop(columns=['EIN', 'NAME'])
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
app_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
app_types = app_df['APPLICATION_TYPE'].value_counts()
app_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_types[app_types < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
app_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classes = app_df['CLASSIFICATION'].value_counts()
classes

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classes[classes > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classes[classes < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
categories = app_df.dtypes[app_df.dtypes == "object"].index.tolist()
dummied_df = pd.get_dummies(app_df[categories])
dummied_df = dummied_df.merge(app_df[app_df.columns.difference(categories)], left_index=True, right_index=True)
dummied_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT,IS_SUCCESSFUL,STATUS
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,5000,1,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,108590,1,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,5000,0,1
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,6692,1,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,142590,1,1


In [12]:
# Split our preprocessed data into our features and target arrays
y = dummied_df['IS_SUCCESSFUL'].values
X = dummied_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimize, Compile, Train and Evaluate the Model

In [14]:
# import the keras tuner modules
!pip install -q keras-tuner

import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.1 MB/s eta 0:00:00


In [15]:
# Define the model (in function) - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def make_model(hp):
  # Set up Sequential Model
  nn_model = tf.keras.models.Sequential()
  input_dim = len(X_train_scaled[0])

  # Set up Choices for activation functions for each hidden layer
  activations = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

  # set up the keras tuner to decide (loop) through a number of attributes in the first layer - input_dim = 2
  # loop through 1 - 10 neurons on the second layer

  nn_model.add(tf.keras.layers.Dense(
                                      input_dim=input_dim,
                                      activation=activations,
                                      units=hp.Int('first_units', min_value=1, max_value=10)))

   # set up the keras tuner to test different hidden layers
  for num in range(hp.Int('num_layers', 1, 6)):
    nn_model.add(
          tf.keras.layers.Dense(activation=activations,
                                units=hp.Int('units_' + str(num),
                                             min_value=1,
                                             max_value=10)))
  # set up output layer
  nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  #compile model
  nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return nn_model


In [16]:
# use Hyperband function to run optimizer function
tuner_model = kt.Hyperband(
    # pass model function
    make_model,
    #pass objective function - 'val_accuracy'
    objective='val_accuracy',
    max_epochs=25,
    hyperband_iterations=3
)

In [ ]:
# search tuner_model for best combo
tuner_model.search(X_train_scaled, y_train, epochs=25, validation_data=[X_test_scaled, y_test])

Trial 88 Complete [00h 01m 07s]
val_accuracy: 0.7292128205299377

Best val_accuracy So Far: 0.7338775396347046
Total elapsed time: 00h 42m 22s

Search: Running Trial #89

Value             |Best Value So Far |Hyperparameter
relu              |tanh              |activation
8                 |5                 |first_units
5                 |6                 |num_layers
4                 |5                 |units_0
8                 |2                 |units_1
7                 |3                 |units_2
8                 |7                 |units_3
5                 |8                 |units_4
10                |10                |units_5
25                |25                |tuner/epochs
0                 |9                 |tuner/initial_epoch
0                 |1                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/25


In [ ]:
# use .get_best_hyperparameters
tuner_model.get_best_hyperparameters(1)[0].values

In [ ]:
# use get_best_models to extract the hyper parameters and evaluate the model loss and accuracy
best_model = tuner_model.get_best_models(1)[0]

model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test)

In [ ]:
# create folder to hold final model
import os

os.makedirs("model_checkpoints/", exist_ok=True)


In [ ]:
# Evaluate the model using the test data
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
best_model.save("model_checkpoints/alphabet_soup_charity_optimized.h5")